In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['pdf.fonttype'] = 42

## Figure 3a

In [ ]:
from src.DMS_Profile import Genome, Targeted_DMS_Profile, targeted_combine_profiles, HDP

In [ ]:
# load rrna sequence
genome_rrna = Genome('../data/genome/scer_chr_kan.FASTA', coords=(11, 451575, 458433), reverse=True)

# ribosomal RNA secondary structure
dbr_18 = '...((((.........))))((((.(((((((.(((((((((.....(((.(((..((...(((..(.((..........)))..))))).....((((.......(((((((..((..(((((((............(((((....(((((((.....)))))))....)))))......(((((((((....)))))))))(((.(((((((.......(((((.(((....)))...))))).....))))))).)..))...((((.((((.....))))))))..))))))).))))))))).(((..(.(((....((((((((.......))))))))))).....))))...((((((((....))))...))))))))((((((..........)))))).((((....))))...)))))))......(.(((...(((((...))))).)))).)).))))))....((((..(((((((....)))))))..).))).....((((((((.......))))))))........((.((......(.((((((..(((....)))....))))))))).)).))))))))))).....(...(((.......((((...(((.((....((((((((((((.((((.(((.....)))...)))).....))))))))))))....((((((....(((((((((.....)))))))))....))))))..(((((((((.......(((..(.(...).)..(((.....)))...)))......)))))..)))).....(.((....(.((.(((.............))).))..)..)).)..))...((((((((((.((((((((((((((((((((...(((......)))......))))))))))))....(..((....)))))))))))))))).))))..))))...)))).(..((((((...(((.(((((.........))))).)))))))))..).......((((((.(((..(((((((...((..........)))))))))..)))...((....))...)))....))).))))(((((.((.((((....)))))))))))........(((((.(((((((..((..(((((((((((((((((.(.)((((........))))........(((((((....(((((....(((((((((..........)))))))))..))))).(.((.((((..((((((((((..(((((((((....)))..((((......))))..)))))).....((((((((.((((..(((((.((((((....))))))...)))))..))))))).((.(((((((...)))))))))....)))))...))))).)))...).))))))))....)))))))...)).)))))))))((..(((((((.(...(((.....(((.((((....)))).)))....)))....).)))))))....).((((((((((((........))))))))))))..).))))))(...(((((((((.......)))))))))..)..))...)))))))))).))....((.((...(((((((((((.((((((((((((..(((((((((((((((((((((((((((....)))))))))))))))))))))))))))..)))))))))))))))))))))))....))..))....((((((((((....))))))))))........'
dbr_58 = '........................................((((((((......)))).....((((((....................)))).))...))))...(((...)))(((((((((....))))))))).....................'
dbr_25 = '..........................((((((.....((....))......))))).).........((....))..(((((......((.....)).....))))).((((..((((((((((((((((((...))))))))))))))))))....(((((((.((((((((((((((((((((...(.(((((....))))))..(...).(((.......)))....))))))))..))))))))....)))))))))))........(((((((........))))))).....(((((........)))))..)))).................................((....))...............((((....))))..................................((((((((((((.(..(((((...((((..((((((((.....))))))))......))))...))))).(((((((((((((((((((((((......((((((((..((((((((((....))))))..)))).))))))))....))))).))))))))))).......(((....))).....)))))))..).))))))))))))..((((........))))((((((....(((((((((((((..((((..((((.....))))..))))...((....)).......(((((..(((((....)))))...))))).((((((((...((....))...))).))).))....)))))))))))))..((((...((((((...((((((((((((.((((...(((((((....)))))))....))))...((((.((.((........))))))))..).))))))))))).(((....))).......))))))......))))..((((((....(((((....))))).(((((((((.......((((((((..(.(((....((((((((((..((((((((.....))))))))))))))))))..)).).)..)))))))).(((((((((.........)))))))))...........)))))))).)...((((((((((.....))))))))))....(((......)))...(((((((...(((......(.((((((.................(((.((((((((((.......(((((.(((..(((.........)))..)))(...)(...((......)).).)))))...)))))))))))))...................))))))))))...)))))))(((((((((.........)))))))))....))))))(.((((((((.((......((((.((((....))))))))...))))))))))..).....))))))((((.((((.....((((((.......))))))((((.....(((((.....(((((((...((.....)).))))))).....(((.((...(((((.(((....))))))......(((((........))))).....))....))...(((((........)))))..((((.(((((((.....(((....))).......((((((...((((.(((((((.(((((((((.........))))))...(((((((..(((((((((((((.....)))))))))))))..)).)))))......(((((((....)))))))..)))...))))))).)))).........))))))............))))))))))).))).........((......))))))).((((.....))))..))))..((((....((((((((((((..............(((((...((....))....)))))((((((((((((((.((((((((((((((((((....(((((((((((((((((((.(((((((((((((....))))))))))))).))))))))))))).))))))....))))))))))(((((.....)))))....))))))))......)))))..)))))))))(...).......(((((((((..(((.........)))..(((((((((..((((.....))))..)..))))))))..((((.(((..((((((.(..(((....(((((....)))))....))))))))..)))))((((((.......))))))(...).((((.(...(.)).(((((((.)..))))))..)..)))..))))...)).)))))))....)))))))))))).))))))))))))....(((((........)))))..((((((((.(((.((.....(((((.(((((((.(((((..(((((((..((((((((((((((((((.........(((((..((....))............))))))))))))).))))))))))...(((((.....((((..(((((......)))))...............((((((....))))))..))))..))))).....)))))))...)))))..((((.....))).)((((((...........))))))..((..((((((((((.((((((.......))))))...))((......))....))))))))..(((((((((.......))))))))).(((....))).))..........((((((..((((.......)))))))))).........)))))..))))))).......(((..(((((((...((.......))...)))))))...)))..........(((((((((((..((((((((((....))))))))..)).(((((.....))))).....)))))..)..)))).).....(((((((....))).))))....))..)))))))))))..((((((((..((((.((((((((...............))))))))(((((...(((....(((((((((((....))))))..)))))...)))..)))))..(((....(((((..........)))))....))).))))....)))))))).......(((((((((((((................((((((.....))))))...............((((((((..(((((((((((......))))))))))))))))))).............)))))))))))))..((((((((((.......)))))....(((((((((.((..(..((((.((((((((......)))))))))))))...))....))).))))))..)))))..'
dbr_full = 701*'?' + dbr_18 + 361*'?' + dbr_58 + 232*'?' + dbr_25 + 210*'?'
mask_dat = np.array([True if i in [')', '(', '.'] else False for i in dbr_full])

# load CoSTseq data
wild_nas_1 = Targeted_DMS_Profile('../data/pro_wild_1_rRNA_agg.pkl', 'wild_R1', genome_rrna, reverse=True) 
wild_nas_2 = Targeted_DMS_Profile('../data/pro_wild_2_rRNA_agg.pkl', 'wild_R2', genome_rrna, reverse=True)
wild_nas_3 = Targeted_DMS_Profile('../data/pro_wild_3_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas = targeted_combine_profiles([wild_nas_1, wild_nas_2, wild_nas_3], 'wild', genome_rrna, per_exclude=10)

dbp7_nas_1 = Targeted_DMS_Profile('../data/pro_dbp7_1_rRNA_agg.pkl', 'dbp7_R1', genome_rrna, reverse=True) 
dbp7_nas_2 = Targeted_DMS_Profile('../data/pro_dbp7_2_rRNA_agg.pkl', 'dbp7_R2', genome_rrna, reverse=True)
dbp7_nas_3 = Targeted_DMS_Profile('../data/pro_dbp7_3_rRNA_agg.pkl', 'dbp7_R3', genome_rrna, reverse=True)
dbp7_nas = targeted_combine_profiles([dbp7_nas_1, dbp7_nas_2, dbp7_nas_3], 'dbp7', genome_rrna, per_exclude=10)

# load HDProbe data
dbp7_hdp   = HDP('../data/mat_dbp7_mrna.csv', 'dbp7', thresh_col='padj_BH')
dbp7_hdp_r = HDP('../data/pro_dbp7_rrna.csv', 'dbp7', thresh_col='padj_BF')
dbp3_hdp   = HDP('../data/mat_dbp3_mrna.csv', 'dbp3', thresh_col='padj_BH')
dbp3_hdp_r = HDP('../data/pro_dbp3_rrna.csv', 'dbp3', thresh_col='padj_BF')
spt4_hdp   = HDP('../data/mat_spt4_mrna.csv', 'spt4', thresh_col='padj_BH')
spt4_hdp_r = HDP('../data/pro_spt4_rrna.csv', 'spt4', thresh_col='padj_BF')

In [ ]:
#draw figure
fig, axs = plt.subplots(2, 1, figsize=[18, 3])
plt.subplots_adjust(hspace=0.3)

coords = (3547-100, 3547+100)

# mark significantly different nucleotides
sig_dbp7 = dbp7_hdp_r.hdp[np.abs(dbp7_hdp_r.hdp['difference']) >= 0.8]['position']
sig_string_nas = ''.join(['*' if pos+1 in list(sig_dbp7) else ' ' for pos in range(len(genome_rrna.seq))])

# wildtype
x, y, norm_vec = wild_nas.plot_profile(coords[0], coords[1], genome_rrna, axs[0], cmap_loc='../cmap.txt')
axs[0].text(0.005, 0.82, 'wildtype nascent', transform=axs[0].transAxes)
axs[0].set_xticks(np.arange(coords[0], coords[1], 1))
axs[0].set_xticklabels(sig_string_nas[coords[0]:coords[1]], fontname='Courier New', fontsize=8, weight='bold')

# dbp7∆
x, y, norm_vec = dbp7_nas.plot_profile(coords[0], coords[1], genome_rrna, axs[1], cmap_loc='../cmap.txt')
axs[1].text(0.005, 0.82, 'dbp7∆ nascent', transform=axs[1].transAxes)

plt.setp(axs, ylim=(0, 4), ylabel='DMS reactivity')
plt.show()

## Figure 3b

In [ ]:
from src.CotrxMatrix import CotrxMatrix, combine_reps
from src.dms_utils import weighted_moving_average

In [ ]:
# load co-transcriptional matrices

wild_1 = CotrxMatrix('../data/pro_wild_1_rRNA.pkl', 'wild_R1', genome_rrna.seq, '-')
wild_2 = CotrxMatrix('../data/pro_wild_2_rRNA.pkl', 'wild_R2', genome_rrna.seq, '-')
wild_3 = CotrxMatrix('../data/pro_wild_3_rRNA.pkl', 'wild_R3', genome_rrna.seq, '-')
wild_7 = CotrxMatrix('../data/pro_wild_7_rRNA.pkl', 'wild_R7', genome_rrna.seq, '-')
wild_8 = CotrxMatrix('../data/pro_wild_8_rRNA.pkl', 'wild_R8', genome_rrna.seq, '-')
wild = combine_reps([wild_1, wild_2, wild_3, wild_7, wild_8], 'wild') # reps 4, 5, 6 were noisy in this area (but Pol I average data is fine), therefore not included here. All other mutants agree with these wt profiles, so therefore it's okay to exclude the noisy ones here.
wild.process()

dbp7_1 = CotrxMatrix('../data/pro_dbp7_1_rRNA.pkl', 'dbp7_R1', genome_rrna.seq, '-')
dbp7_2 = CotrxMatrix('../data/pro_dbp7_2_rRNA.pkl', 'dbp7_R2', genome_rrna.seq, '-')
dbp7_3 = CotrxMatrix('../data/pro_dbp7_3_rRNA.pkl', 'dbp7_R3', genome_rrna.seq, '-')
dbp7 = combine_reps([dbp7_1, dbp7_2, dbp7_3], 'dbp7')
dbp7.process()

dbp3_1 = CotrxMatrix('../data/pro_dbp3_1_rRNA.pkl', 'dbp3_R1', genome_rrna.seq, '-')
dbp3_2 = CotrxMatrix('../data/pro_dbp3_2_rRNA.pkl', 'dbp3_R2', genome_rrna.seq, '-')
dbp3_3 = CotrxMatrix('../data/pro_dbp3_3_rRNA.pkl', 'dbp3_R3', genome_rrna.seq, '-')
dbp3 = combine_reps([dbp3_1, dbp3_2, dbp3_3], 'dbp3')
dbp3.process()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15,2), sharey=True, sharex=True)

for i, res in enumerate([3547, 3569, 3570, 3574]):
    x_1, y_1 = wild.get_residue_txn(res, min_cov=800, binned=False)
    z_1 = weighted_moving_average(x_1, y_1, 10)

    x_2, y_2 = dbp7.get_residue_txn(res, min_cov=800, binned=False)
    z_2 = weighted_moving_average(x_2, y_2, 5)

    axs[i].plot(x_1[x_1<3750], y_1[x_1<3750], 'ok', ms=5, label='wildtype')
    axs[i].plot(x_1[x_1<3750], z_1[x_1<3750], '-k', lw=2)
    axs[i].plot(x_2[x_2<3750], y_2[x_2<3750], 'og', ms=5, label='dbp7∆')
    axs[i].plot(x_2[x_2<3750], z_2[x_2<3750], '-g', lw=2)
    axs[i].set_title(f"Pol I at {res+1}")

axs[0].legend()
plt.setp(axs, ylim=[0, 3], xlim=[3550,3750])
plt.show()

## Figure 3c

In [ ]:
from tqdm import tqdm
from scipy.interpolate import interp1d
from src.dms_utils import moving_average

In [ ]:
# difference in trajectories for a range of positions behind polymerase
def compare_trajectories(mat_1, mat_2, pos_diff, min_n=30, avg_win=30):

    diffs_avg_all = []
    for pos_diff in tqdm(pos_diff): # range of polymerase positions that should be included in the comparison
        diffs = []
        resis = []
        for res in range(len(genome_rrna.seq)):
            if genome_rrna.seq[res] in ['A', 'C', 'U']:

                # get trajectories
                x_1, y_1 = mat_1.get_residue_txn(res, min_cov=800, binned=False)
                x_2, y_2 = mat_2.get_residue_txn(res, min_cov=800, binned=False)

                if x_1.size >= min_n and y_1.size >= min_n:
                    # calculate moving average for trajectories
                    z_1 = weighted_moving_average(x_1, y_1, 7)
                    z_2 = weighted_moving_average(x_2, y_2, 7)

                    # interpolate on moving average
                    fit_1 = interp1d(x_1, z_1)
                    xin_1 = np.arange(x_1[0], x_1[-1]+1)
                    int_1 = fit_1(xin_1)

                    fit_2 = interp1d(x_2, z_2)
                    xin_2 = np.arange(x_2[0], x_2[-1]+1)
                    int_2 = fit_2(xin_2)

                    # if the requested nucleotide position is in range, calculate the difference
                    if int_1[xin_1==res+pos_diff].size == 1 and int_2[xin_2==res+pos_diff].size == 1:
                        resis.append(res)
                        diffs.append(float(int_1[xin_1==(res+pos_diff)])-float(int_2[xin_2==(res+pos_diff)]))
                    else:
                        resis.append(res)
                        diffs.append(np.nan)
                else:
                    resis.append(res)
                    diffs.append(np.nan)

        # calculate moving average of difference vector
        diffs_avg = moving_average(diffs, avg_win)
        diffs_avg_all.append(diffs_avg)
    
    return(diffs_avg_all, resis)

# slightly affecting the outcome in different ways:
# - sliding window size
# - which Pol positions to include
# - min number of data points in trajectory

In [ ]:
wild_dbp7, resis = compare_trajectories(wild, dbp7, np.arange(80, 151, 1))
wild_dbp3, resis = compare_trajectories(wild, dbp3, np.arange(80, 151, 1))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 6), sharex=True, sharey=True)

for ax, diffs_avg_all in zip(axs, [wild_dbp7, wild_dbp3]):
    ax.fill_between(np.arange(0, len(mask_dat)), mask_dat, color='lightgray')
    ax.fill_between(resis, np.nanmedian(diffs_avg_all, axis=0), color='red')
    for i in diffs_avg_all:
        ax.plot(resis, i, '-g', alpha=0.05)
        ax.plot([0, 6600], [0, 0], '--k')
    
plt.setp(axs, ylim=[-0.5, 0.5])
plt.show()

## Figure S3b

In [ ]:
# load HDProbe data
dbp7_hdp_r = HDP('../data/pro_dbp7_rrna.csv', 'dbp7', thresh_col='padj_BF')

# plot
fig, axs = plt.subplots(figsize=(3, 3))
dbp7_hdp_r.erupt(axs, cmap_loc='../cmap.txt', alpha=0.5, edgecolors='none')

## Figure S3c

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15,2), sharey=True, sharex=True)

for i, res in enumerate([3547, 3569, 3570, 3574]):
    x_1, y_1 = wild.get_residue_txn(res, min_cov=800, binned=False)
    z_1 = weighted_moving_average(x_1, y_1, 10)

    x_3, y_3 = dbp3.get_residue_txn(res, min_cov=800, binned=False)
    z_3 = weighted_moving_average(x_3, y_3, 5)

    axs[i].plot(x_1[x_1<3750], y_1[x_1<3750], 'ok', ms=5, label='wildtype')
    axs[i].plot(x_1[x_1<3750], z_1[x_1<3750], '-k', lw=2)
    axs[i].plot(x_3[x_3<3750], y_3[x_3<3750], 'og', ms=5, label='dbp3∆')
    axs[i].plot(x_3[x_3<3750], z_3[x_3<3750], '-g', lw=2)
    axs[i].set_title(f"Pol I at {res+1}")

axs[0].legend()
plt.setp(axs, ylim=[0, 3], xlim=[3550,3750])
plt.show()

## Figure S3d

In [ ]:
# load HDProbe data
dbp7_hdp   = HDP('../data/mat_dbp7_mrna.csv', 'dbp7', thresh_col='padj_BH')

# plot
fig, axs = plt.subplots(figsize=(3, 3))
dbp7_hdp.erupt(axs, cmap_loc='../cmap.txt', alpha=0.5, edgecolors='none')

## Figure S3e

In [ ]:
from src.DMS_Profile import DMS_Profile, combine_profiles, Annotation

In [ ]:
# load genome and annotation
genome = Genome('../data/genome/scer_chr_kan.FASTA')
annotation = Annotation('../data/genome/scer_kan.gff')

In [ ]:
# load mRNA DMS-MaPseq data
mat_wild_1 = DMS_Profile('../data/mat_dms/mat_wild_1_mRNA_agg.pkl', 's', genome)
mat_wild_2 = DMS_Profile('../data/mat_dms/mat_wild_2_mRNA_agg.pkl', 's', genome)
mat_wild_3 = DMS_Profile('../data/mat_dms/mat_wild_3_mRNA_agg.pkl', 's', genome)
mat_wild   = combine_profiles([mat_wild_1, mat_wild_2, mat_wild_3], 'wild', genome, min_cov=800)

mat_dbp7_1 = DMS_Profile('../data/mat_dms/mat_dbp7_1_mRNA_agg.pkl', 's', genome)
mat_dbp7_2 = DMS_Profile('../data/mat_dms/mat_dbp7_2_mRNA_agg.pkl', 's', genome)
mat_dbp7_3 = DMS_Profile('../data/mat_dms/mat_dbp7_3_mRNA_agg.pkl', 's', genome)
mat_dbp7   = combine_profiles([mat_dbp7_1, mat_dbp7_2, mat_dbp7_3], 'dbp7', genome, min_cov=800)

In [ ]:
# annotate hits and find their relative positions
dbp7_hdp.annotate_hits(annotation)
binsize=0.075
bins_hit, hist_hit, bins_cov_dbp7, hist_cov_dbp7, rela_all, anno_all = dbp7_hdp.get_hdp_meta(annotation, mat_dbp7.cov_p, mat_dbp7.cov_m, sig_threshold=0.05, binsize=0.075, n_bins_cov=100)
_       ,        _, bins_cov_wild, hist_cov_wild, _       , _        = dbp7_hdp.get_hdp_meta(annotation, mat_wild.cov_p, mat_wild.cov_m, sig_threshold=0.05, binsize=0.075, n_bins_cov=100)

In [ ]:
# draw figure
fig, ax1 = plt.subplots(figsize=(5,3))

ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax1.plot(bins_hit, hist_hit, '-o')
ax2.plot(bins_cov_dbp7, hist_cov_dbp7, '-k')
ax3.plot(bins_cov_wild, hist_cov_wild, '-g')
ax1.set_xlabel('relative position in transcript')
plt.ylabel('Median coverage')
ax1.set_ylabel('Number of HDP hits')
# plt.savefig('figs_raw/hdp_mRNA_dbp7_meta.pdf')  
plt.show()